In [1]:
from numpy import array
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from fitter import Fitter

from sklearn.model_selection import train_test_split
import statsmodels.api as sm

from sklearn import metrics
from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import SpectralClustering
from sklearn import tree

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

import time

import synthgauge as sg

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn import svm

from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.preprocessing import PolynomialFeatures

from sklearn import metrics
from sklearn.metrics import r2_score

In [2]:
crit = pd.read_excel('C:/Users/ursuz/OneDrive/Escritorio/Tesis/critica.xlsx', index_col=0)
sale = pd.read_excel('C:/Users/ursuz/OneDrive/Escritorio/Tesis/vgsales.xlsx', index_col=0)

crit['name'] = crit['name'].str.lower()
sale['name'] = sale['name'].str.lower()

baseuni = pd.merge(crit, sale, on = ["name","platform"])
baseuni

,name,platform,release_date,summary,meta_score,user_review,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,the legend of zelda: ocarina of time,N64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ...",99.0,9.1,1998,Action,Nintendo,4.10,1.89,1.45,0.16,7.60
1,tony hawk's pro skater 2,PS,"September 20, 2000",As most major publishers' development efforts ...,98.0,7.4,2000,Sports,Activision,3.05,1.41,0.02,0.20,4.68
2,grand theft auto iv,PS3,"April 29, 2008",[Metacritic's 2008 PS3 Game of the Year; Also ...,98.0,7.7,2008,Action,Take-Two Interactive,4.76,3.76,0.44,1.62,10.57
3,soulcalibur,DC,"September 8, 1999","This is a tale of souls and swords, transcendi...",98.0,8.4,1999,Fighting,Namco Bandai Games,0.00,0.00,0.34,0.00,0.34
4,grand theft auto v,PS3,"September 17, 2013","Los Santos is a vast, sun-soaked metropolis fu...",97.0,8.3,2013,Action,Take-Two Interactive,7.01,9.27,0.97,4.14,21.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3433,batman: dark tomorrow,XB,"March 18, 2003","Batman, the Dark Knight detective of Gotham Ci...",25.0,3.3,2003,Action,Kemco,0.08,0.02,0.00,0.00,0.11
3434,gravity games bike: street vert dirt,PS2,"June 27, 2002",Gravity Games Bike: Street. Vert. Dirt. delive...,24.0,3.4,2002,Sports,Midway Games,0.05,0.04,0.00,0.01,0.10
3435,legends of wrestling ii,GBA,"November 25, 2002",Legends of Wrestling II takes you through the ...,24.0,3.6,2002,Fighting,Acclaim Entertainment,0.05,0.02,0.00,0.00,0.07
3436,pulse racer,XB,"January 2, 2003",Pulse Racer takes you to a future where racers...,24.0,2.2,2003,Racing,Jaleco,0.01,0.00,0.00,0.00,0.02


In [4]:
baseNA = pd.DataFrame()
baseNA = baseNA.assign(Plataforma=baseuni['platform'], Ventas_norteamerica=baseuni['NA_Sales'], Calificacion_critica=baseuni['meta_score'])
baseNA

,Plataforma,Ventas_norteamerica,Calificacion_critica
0,N64,4.10,99.0
1,PS,3.05,98.0
2,PS3,4.76,98.0
3,DC,0.00,98.0
4,PS3,7.01,97.0
...,...,...,...
3433,XB,0.08,25.0
3434,PS2,0.05,24.0
3435,GBA,0.05,24.0
3436,XB,0.01,24.0


In [5]:
baseEU = pd.DataFrame()
baseEU = baseEU.assign(Plataforma=baseuni['platform'], Ventas_europeas=baseuni['EU_Sales'], Calificacion_critica=baseuni['meta_score'])
baseEU

,Plataforma,Ventas_europeas,Calificacion_critica
0,N64,1.89,99.0
1,PS,1.41,98.0
2,PS3,3.76,98.0
3,DC,0.00,98.0
4,PS3,9.27,97.0
...,...,...,...
3433,XB,0.02,25.0
3434,PS2,0.04,24.0
3435,GBA,0.02,24.0
3436,XB,0.00,24.0


In [6]:
baseJP= pd.DataFrame()
baseJP = baseJP.assign(Plataforma=baseuni['platform'], Ventas_japonesas=baseuni['JP_Sales'], Calificacion_critica=baseuni['meta_score'])
baseJP

,Plataforma,Ventas_japonesas,Calificacion_critica
0,N64,1.45,99.0
1,PS,0.02,98.0
2,PS3,0.44,98.0
3,DC,0.34,98.0
4,PS3,0.97,97.0
...,...,...,...
3433,XB,0.00,25.0
3434,PS2,0.00,24.0
3435,GBA,0.00,24.0
3436,XB,0.00,24.0


In [9]:
valores_unicosNA = baseNA['Plataforma'].unique()
valores_unicosNA

array(['N64', 'PS', 'PS3', 'DC', 'PS2', 'PS4', 'GC', 'XB', 'GBA', 'PSV'],
      dtype=object)

In [82]:
condicion = baseNA['Plataforma'] == 'N64'
N64NA = baseNA[condicion]
N64NA

,Plataforma,Ventas_norteamerica,Calificacion_critica
0,N64,4.10,99.0
6,N64,1.55,97.0
12,N64,5.80,96.0
29,N64,1.90,95.0
55,N64,6.91,94.0
77,N64,0.58,93.0
93,N64,1.87,92.0
110,N64,1.98,92.0
114,N64,0.53,92.0
130,N64,0.78,91.0


In [11]:
condicion = baseNA['Plataforma'] == 'PS'
PSNA = baseNA[condicion]
PSNA

,Plataforma,Ventas_norteamerica,Calificacion_critica
1,PS,3.05,98.0
16,PS,3.27,96.0
20,PS,4.02,96.0
35,PS,1.62,94.0
41,PS,0.62,94.0
...,...,...,...
3360,PS,0.03,47.0
3388,PS,0.04,37.0
3425,PS,0.12,28.0
3431,PS,0.12,26.0


In [12]:
condicion = baseNA['Plataforma'] == 'PS3'
PS3NA = baseNA[condicion]
PS3NA

,Plataforma,Ventas_norteamerica,Calificacion_critica
2,PS3,4.76,98.0
4,PS3,7.01,97.0
13,PS3,3.27,96.0
15,PS3,2.70,96.0
24,PS3,2.80,95.0
...,...,...,...
3408,PS3,0.04,35.0
3409,PS3,0.28,34.0
3412,PS3,0.06,33.0
3421,PS3,0.05,30.0


In [13]:
condicion = baseNA['Plataforma'] == 'DC'
DCNA = baseNA[condicion]
DCNA

,Plataforma,Ventas_norteamerica,Calificacion_critica
3,DC,0.00,98.0
11,DC,1.02,97.0
71,DC,0.00,93.0
196,DC,0.00,90.0
228,DC,0.00,89.0
248,DC,0.00,89.0
275,DC,0.00,88.0
586,DC,0.00,84.0
667,DC,0.00,83.0
820,DC,0.00,82.0


In [14]:
condicion = baseNA['Plataforma'] == 'PS2'
PS2NA = baseNA[condicion]
PS2NA

,Plataforma,Ventas_norteamerica,Calificacion_critica
5,PS2,2.66,97.0
9,PS2,6.99,97.0
18,PS2,2.08,96.0
21,PS2,2.45,96.0
22,PS2,9.43,95.0
...,...,...,...
3404,PS2,0.11,35.0
3407,PS2,0.07,35.0
3414,PS2,0.00,32.0
3427,PS2,0.04,27.0


In [15]:
condicion = baseNA['Plataforma'] == 'PS4'
PS4NA = baseNA[condicion]
PS4NA

,Plataforma,Ventas_norteamerica,Calificacion_critica
7,PS4,3.80,97.0
62,PS4,0.00,93.0
70,PS4,1.08,93.0
75,PS4,1.30,93.0
90,PS4,1.00,92.0
...,...,...,...
3318,PS4,0.07,56.0
3324,PS4,0.04,55.0
3372,PS4,0.00,42.0
3417,PS4,0.09,32.0


In [16]:
condicion = baseNA['Plataforma'] == 'GC'
GCNA = baseNA[condicion]
GCNA

,Plataforma,Ventas_norteamerica,Calificacion_critica
8,GC,1.96,97.0
14,GC,0.98,96.0
17,GC,1.15,96.0
19,GC,2.60,96.0
53,GC,0.40,94.0
...,...,...,...
3398,GC,0.09,36.0
3401,GC,0.03,35.0
3423,GC,0.06,29.0
3426,GC,0.01,27.0


In [17]:
condicion = baseNA['Plataforma'] == 'XB'
XBNA = baseNA[condicion]
XBNA

,Plataforma,Ventas_norteamerica,Calificacion_critica
10,XB,4.98,97.0
27,XB,6.82,95.0
44,XB,0.07,94.0
47,XB,0.84,94.0
52,XB,1.68,94.0
...,...,...,...
3420,XB,0.04,30.0
3424,XB,0.07,28.0
3430,XB,0.02,26.0
3433,XB,0.08,25.0


In [18]:
condicion = baseNA['Plataforma'] == 'GBA'
GBANA = baseNA[condicion]
GBANA

,Plataforma,Ventas_norteamerica,Calificacion_critica
28,GBA,1.75,95.0
32,GBA,0.85,95.0
96,GBA,0.50,92.0
119,GBA,1.18,92.0
120,GBA,0.00,92.0
...,...,...,...
3405,GBA,0.02,35.0
3411,GBA,0.07,33.0
3422,GBA,0.10,30.0
3429,GBA,0.05,26.0


In [19]:
condicion = baseNA['Plataforma'] == 'PSV'
PSVNA = baseNA[condicion]
PSVNA

,Plataforma,Ventas_norteamerica,Calificacion_critica
298,PSV,0.13,88.0
326,PSV,0.40,88.0
362,PSV,0.03,87.0
367,PSV,0.17,87.0
416,PSV,0.21,86.0
...,...,...,...
3113,PSV,0.08,59.0
3196,PSV,0.13,58.0
3211,PSV,0.00,58.0
3304,PSV,0.00,57.0


In [20]:
valores_unicosEU = baseEU['Plataforma'].unique()
valores_unicosEU

array(['N64', 'PS', 'PS3', 'DC', 'PS2', 'PS4', 'GC', 'XB', 'GBA', 'PSV'],
      dtype=object)

In [80]:
condicion = baseEU['Plataforma'] == 'N64'
N64EU = baseEU[condicion]
N64EU

,Plataforma,Ventas_europeas,Calificacion_critica
0,N64,1.89,99.0
6,N64,0.75,97.0
12,N64,2.01,96.0
29,N64,0.67,95.0
55,N64,2.85,94.0
77,N64,0.18,93.0
93,N64,1.13,92.0
110,N64,0.58,92.0
114,N64,0.22,92.0
130,N64,0.40,91.0


In [23]:
condicion = baseEU['Plataforma'] == 'PS'
PSEU = baseEU[condicion]
PSEU

,Plataforma,Ventas_europeas,Calificacion_critica
1,PS,1.41,98.0
16,PS,2.22,96.0
20,PS,3.87,96.0
35,PS,0.77,94.0
41,PS,0.42,94.0
...,...,...,...
3360,PS,0.02,47.0
3388,PS,0.03,37.0
3425,PS,0.08,28.0
3431,PS,0.08,26.0


In [24]:
condicion = baseEU['Plataforma'] == 'PS3'
PS3EU = baseEU[condicion]
PS3EU

,Plataforma,Ventas_europeas,Calificacion_critica
2,PS3,3.76,98.0
4,PS3,9.27,97.0
13,PS3,2.25,96.0
15,PS3,1.91,96.0
24,PS3,2.05,95.0
...,...,...,...
3408,PS3,0.03,35.0
3409,PS3,0.17,34.0
3412,PS3,0.00,33.0
3421,PS3,0.00,30.0


In [25]:
condicion = baseEU['Plataforma'] == 'DC'
DCEU = baseEU[condicion]
DCEU

,Plataforma,Ventas_europeas,Calificacion_critica
3,DC,0.00,98.0
11,DC,0.05,97.0
71,DC,0.00,93.0
196,DC,0.00,90.0
228,DC,0.00,89.0
248,DC,0.00,89.0
275,DC,0.00,88.0
586,DC,0.00,84.0
667,DC,0.00,83.0
820,DC,0.00,82.0


In [26]:
condicion = baseEU['Plataforma'] == 'PS2'
PS2EU = baseEU[condicion]
PS2EU

,Plataforma,Ventas_europeas,Calificacion_critica
5,PS2,1.29,97.0
9,PS2,4.51,97.0
18,PS2,0.83,96.0
21,PS2,2.01,96.0
22,PS2,0.40,95.0
...,...,...,...
3404,PS2,0.09,35.0
3407,PS2,0.06,35.0
3414,PS2,0.00,32.0
3427,PS2,0.03,27.0


In [27]:
condicion = baseEU['Plataforma'] == 'PS4'
PS4EU = baseEU[condicion]
PS4EU

,Plataforma,Ventas_europeas,Calificacion_critica
7,PS4,5.81,97.0
62,PS4,0.00,93.0
70,PS4,1.35,93.0
75,PS4,2.07,93.0
90,PS4,0.76,92.0
...,...,...,...
3318,PS4,0.08,56.0
3324,PS4,0.02,55.0
3372,PS4,0.05,42.0
3417,PS4,0.08,32.0


In [28]:
condicion = baseEU['Plataforma'] == 'GC'
GCEU = baseEU[condicion]
GCEU

,Plataforma,Ventas_europeas,Calificacion_critica
8,GC,0.67,97.0
14,GC,0.42,96.0
17,GC,0.36,96.0
19,GC,0.99,96.0
53,GC,0.10,94.0
...,...,...,...
3398,GC,0.02,36.0
3401,GC,0.01,35.0
3423,GC,0.02,29.0
3426,GC,0.00,27.0


In [29]:
condicion = baseEU['Plataforma'] == 'XB'
XBEU = baseEU[condicion]
XBEU

,Plataforma,Ventas_europeas,Calificacion_critica
10,XB,1.30,97.0
27,XB,1.53,95.0
44,XB,0.02,94.0
47,XB,0.43,94.0
52,XB,0.44,94.0
...,...,...,...
3420,XB,0.01,30.0
3424,XB,0.02,28.0
3430,XB,0.01,26.0
3433,XB,0.02,25.0


In [30]:
condicion = baseEU['Plataforma'] == 'GBA'
GBAEU = baseEU[condicion]
GBAEU

,Plataforma,Ventas_europeas,Calificacion_critica
28,GBA,0.52,95.0
32,GBA,0.31,95.0
96,GBA,0.19,92.0
119,GBA,0.27,92.0
120,GBA,0.00,92.0
...,...,...,...
3405,GBA,0.01,35.0
3411,GBA,0.03,33.0
3422,GBA,0.04,30.0
3429,GBA,0.02,26.0


In [31]:
condicion = baseEU['Plataforma'] == 'PSV'
PSVEU = baseEU[condicion]
PSVEU

,Plataforma,Ventas_europeas,Calificacion_critica
298,PSV,0.35,88.0
326,PSV,0.72,88.0
362,PSV,0.12,87.0
367,PSV,0.39,87.0
416,PSV,0.31,86.0
...,...,...,...
3113,PSV,0.00,59.0
3196,PSV,0.06,58.0
3211,PSV,0.00,58.0
3304,PSV,0.30,57.0


In [32]:
valores_unicosJP = baseJP['Plataforma'].unique()
valores_unicosJP

array(['N64', 'PS', 'PS3', 'DC', 'PS2', 'PS4', 'GC', 'XB', 'GBA', 'PSV'],
      dtype=object)

In [81]:
condicion = baseJP['Plataforma'] == 'N64'
N64JP = baseJP[condicion]
N64JP

,Plataforma,Ventas_japonesas,Calificacion_critica
0,N64,1.45,99.0
6,N64,0.16,97.0
12,N64,0.13,96.0
29,N64,0.73,95.0
55,N64,1.91,94.0
77,N64,0.59,93.0
93,N64,0.55,92.0
110,N64,0.34,92.0
114,N64,0.00,92.0
130,N64,1.06,91.0


In [34]:
condicion = baseJP['Plataforma'] == 'PS'
PSJP = baseJP[condicion]
PSJP

,Plataforma,Ventas_japonesas,Calificacion_critica
1,PS,0.02,98.0
16,PS,1.40,96.0
20,PS,2.54,96.0
35,PS,2.78,94.0
41,PS,0.69,94.0
...,...,...,...
3360,PS,0.00,47.0
3388,PS,0.00,37.0
3425,PS,0.00,28.0
3431,PS,0.00,26.0


In [35]:
condicion = baseJP['Plataforma'] == 'PS3'
PS3JP = baseJP[condicion]
PS3JP

,Plataforma,Ventas_japonesas,Calificacion_critica
2,PS3,0.44,98.0
4,PS3,0.97,97.0
13,PS3,0.21,96.0
15,PS3,0.11,96.0
24,PS3,0.17,95.0
...,...,...,...
3408,PS3,0.00,35.0
3409,PS3,0.00,34.0
3412,PS3,0.14,33.0
3421,PS3,0.00,30.0


In [36]:
condicion = baseJP['Plataforma'] == 'DC'
DCJP = baseJP[condicion]
DCJP

,Plataforma,Ventas_japonesas,Calificacion_critica
3,DC,0.34,98.0
11,DC,0.00,97.0
71,DC,0.09,93.0
196,DC,0.20,90.0
228,DC,0.20,89.0
248,DC,0.12,89.0
275,DC,0.15,88.0
586,DC,0.13,84.0
667,DC,0.05,83.0
820,DC,0.07,82.0


In [37]:
condicion = baseJP['Plataforma'] == 'PS2'
PS2JP = baseJP[condicion]
PS2JP

,Plataforma,Ventas_japonesas,Calificacion_critica
5,PS2,0.01,97.0
9,PS2,0.30,97.0
18,PS2,0.46,96.0
21,PS2,0.87,96.0
22,PS2,0.41,95.0
...,...,...,...
3404,PS2,0.00,35.0
3407,PS2,0.00,35.0
3414,PS2,0.03,32.0
3427,PS2,0.00,27.0


In [38]:
condicion = baseJP['Plataforma'] == 'PS4'
PS4JP = baseJP[condicion]
PS4JP

,Plataforma,Ventas_japonesas,Calificacion_critica
7,PS4,0.36,97.0
62,PS4,0.34,93.0
70,PS4,0.48,93.0
75,PS4,0.18,93.0
90,PS4,0.26,92.0
...,...,...,...
3318,PS4,0.00,56.0
3324,PS4,0.00,55.0
3372,PS4,0.00,42.0
3417,PS4,0.00,32.0


In [39]:
condicion = baseJP['Plataforma'] == 'GC'
GCJP = baseJP[condicion]
GCJP

,Plataforma,Ventas_japonesas,Calificacion_critica
8,GC,0.10,97.0
14,GC,0.22,96.0
17,GC,0.04,96.0
19,GC,0.89,96.0
53,GC,0.00,94.0
...,...,...,...
3398,GC,0.00,36.0
3401,GC,0.00,35.0
3423,GC,0.00,29.0
3426,GC,0.00,27.0


In [40]:
condicion = baseJP['Plataforma'] == 'XB'
XBJP = baseJP[condicion]
XBJP

,Plataforma,Ventas_japonesas,Calificacion_critica
10,XB,0.08,97.0
27,XB,0.05,95.0
44,XB,0.00,94.0
47,XB,0.00,94.0
52,XB,0.00,94.0
...,...,...,...
3420,XB,0.00,30.0
3424,XB,0.00,28.0
3430,XB,0.00,26.0
3433,XB,0.00,25.0


In [41]:
condicion = baseJP['Plataforma'] == 'GBA'
GBAJP = baseJP[condicion]
GBAJP

,Plataforma,Ventas_japonesas,Calificacion_critica
28,GBA,0.33,95.0
32,GBA,0.00,95.0
96,GBA,0.00,92.0
119,GBA,0.17,92.0
120,GBA,0.28,92.0
...,...,...,...
3405,GBA,0.00,35.0
3411,GBA,0.00,33.0
3422,GBA,0.00,30.0
3429,GBA,0.00,26.0


In [42]:
condicion = baseJP['Plataforma'] == 'PSV'
PSVJP = baseJP[condicion]
PSVJP

,Plataforma,Ventas_japonesas,Calificacion_critica
298,PSV,0.00,88.0
326,PSV,0.02,88.0
362,PSV,0.00,87.0
367,PSV,0.00,87.0
416,PSV,0.28,86.0
...,...,...,...
3113,PSV,0.04,59.0
3196,PSV,0.00,58.0
3211,PSV,0.01,58.0
3304,PSV,0.05,57.0


In [83]:
N64NA = N64NA.dropna()
N64NA = N64NA[(N64NA['Ventas_norteamerica'] != 0)]
N64NA['Ventas_norteamerica'] = N64NA['Ventas_norteamerica'] * 1000000
N64NA

,Plataforma,Ventas_norteamerica,Calificacion_critica
0,N64,4100000.0,99.0
6,N64,1550000.0,97.0
12,N64,5800000.0,96.0
29,N64,1900000.0,95.0
55,N64,6910000.0,94.0
77,N64,580000.0,93.0
93,N64,1870000.0,92.0
110,N64,1980000.0,92.0
114,N64,530000.0,92.0
130,N64,780000.0,91.0


In [84]:
N64EU = N64EU.dropna()
N64EU = N64EU[(N64EU['Ventas_europeas'] != 0)]
N64EU['Ventas_europeas'] = N64EU['Ventas_europeas'] * 1000000
N64EU

,Plataforma,Ventas_europeas,Calificacion_critica
0,N64,1890000.0,99.0
6,N64,750000.0,97.0
12,N64,2010000.0,96.0
29,N64,670000.0,95.0
55,N64,2850000.0,94.0
77,N64,180000.0,93.0
93,N64,1130000.0,92.0
110,N64,580000.0,92.0
114,N64,220000.0,92.0
130,N64,400000.0,91.0


In [85]:
N64JP = N64JP.dropna()
N64JP = N64JP[(N64JP['Ventas_japonesas'] != 0)]
N64JP['Ventas_japonesas'] = N64JP['Ventas_japonesas'] * 1000000
N64JP

,Plataforma,Ventas_japonesas,Calificacion_critica
0,N64,1450000.0,99.0
6,N64,160000.0,97.0
12,N64,130000.0,96.0
29,N64,730000.0,95.0
55,N64,1910000.0,94.0
77,N64,590000.0,93.0
93,N64,550000.0,92.0
110,N64,340000.0,92.0
130,N64,1060000.0,91.0
159,N64,650000.0,91.0


In [46]:
PSNA = PSNA.dropna()
PSNA = PSNA[(PSNA['Ventas_norteamerica'] != 0)]
PSNA['Ventas_norteamerica'] = PSNA['Ventas_norteamerica'] * 1000000
PSNA

,Plataforma,Ventas_norteamerica,Calificacion_critica
1,PS,3050000.0,98.0
16,PS,3270000.0,96.0
20,PS,4020000.0,96.0
35,PS,1620000.0,94.0
41,PS,620000.0,94.0
...,...,...,...
3360,PS,30000.0,47.0
3388,PS,40000.0,37.0
3425,PS,120000.0,28.0
3431,PS,120000.0,26.0


In [47]:
PSEU = PSEU.dropna()
PSEU = PSEU[(PSEU['Ventas_europeas'] != 0)]
PSEU['Ventas_europeas'] = PSEU['Ventas_europeas'] * 1000000
PSEU

,Plataforma,Ventas_europeas,Calificacion_critica
1,PS,1410000.0,98.0
16,PS,2220000.0,96.0
20,PS,3870000.0,96.0
35,PS,770000.0,94.0
41,PS,420000.0,94.0
...,...,...,...
3360,PS,20000.0,47.0
3388,PS,30000.0,37.0
3425,PS,80000.0,28.0
3431,PS,80000.0,26.0


In [48]:
PSJP = PSJP.dropna()
PSJP = PSJP[(PSJP['Ventas_japonesas'] != 0)]
PSJP['Ventas_japonesas'] = PSJP['Ventas_japonesas'] * 1000000
PSJP

,Plataforma,Ventas_japonesas,Calificacion_critica
1,PS,20000.0,98.0
16,PS,1400000.0,96.0
20,PS,2540000.0,96.0
35,PS,2780000.0,94.0
41,PS,690000.0,94.0
46,PS,780000.0,94.0
57,PS,1690000.0,93.0
58,PS,510000.0,93.0
73,PS,210000.0,93.0
97,PS,300000.0,92.0


In [49]:
PS3NA = PS3NA.dropna()
PS3NA = PS3NA[(PS3NA['Ventas_norteamerica'] != 0)]
PS3NA['Ventas_norteamerica'] = PS3NA['Ventas_norteamerica'] * 1000000
PS3NA

,Plataforma,Ventas_norteamerica,Calificacion_critica
2,PS3,4760000.0,98.0
4,PS3,7010000.0,97.0
13,PS3,3270000.0,96.0
15,PS3,2700000.0,96.0
24,PS3,2800000.0,95.0
...,...,...,...
3408,PS3,40000.0,35.0
3409,PS3,280000.0,34.0
3412,PS3,60000.0,33.0
3421,PS3,50000.0,30.0


In [50]:
PS3EU = PS3EU.dropna()
PS3EU = PS3EU[(PS3EU['Ventas_europeas'] != 0)]
PS3EU['Ventas_europeas'] = PS3EU['Ventas_europeas'] * 1000000
PS3EU

,Plataforma,Ventas_europeas,Calificacion_critica
2,PS3,3760000.0,98.0
4,PS3,9270000.0,97.0
13,PS3,2250000.0,96.0
15,PS3,1910000.0,96.0
24,PS3,2050000.0,95.0
...,...,...,...
3397,PS3,10000.0,36.0
3399,PS3,60000.0,36.0
3408,PS3,30000.0,35.0
3409,PS3,170000.0,34.0


In [51]:
PS3JP = PS3JP.dropna()
PS3JP = PS3JP[(PS3JP['Ventas_japonesas'] != 0)]
PS3JP['Ventas_japonesas'] = PS3JP['Ventas_japonesas'] * 1000000
PS3JP

,Plataforma,Ventas_japonesas,Calificacion_critica
2,PS3,440000.0,98.0
4,PS3,970000.0,97.0
13,PS3,210000.0,96.0
15,PS3,110000.0,96.0
24,PS3,170000.0,95.0
...,...,...,...
3277,PS3,20000.0,57.0
3310,PS3,110000.0,57.0
3349,PS3,300000.0,49.0
3393,PS3,10000.0,36.0


In [52]:
DCNA = DCNA.dropna()
DCNA = DCNA[(DCNA['Ventas_norteamerica'] != 0)]
DCNA['Ventas_norteamerica'] = DCNA['Ventas_norteamerica'] * 1000000
DCNA

,Plataforma,Ventas_norteamerica,Calificacion_critica
11,DC,1020000.0,97.0


In [53]:
DCEU = DCEU.dropna()
DCEU = DCEU[(DCEU['Ventas_europeas'] != 0)]
DCEU['Ventas_europeas'] = DCEU['Ventas_europeas'] * 1000000
DCEU

,Plataforma,Ventas_europeas,Calificacion_critica
11,DC,50000.0,97.0


In [54]:
DCJP = DCJP.dropna()
DCJP = DCJP[(DCJP['Ventas_japonesas'] != 0)]
DCJP['Ventas_japonesas'] = DCJP['Ventas_japonesas'] * 1000000
DCJP

,Plataforma,Ventas_japonesas,Calificacion_critica
3,DC,340000.0,98.0
71,DC,90000.0,93.0
196,DC,200000.0,90.0
228,DC,200000.0,89.0
248,DC,120000.0,89.0
275,DC,150000.0,88.0
586,DC,130000.0,84.0
667,DC,50000.0,83.0
820,DC,70000.0,82.0
858,DC,520000.0,82.0


In [55]:
PS2NA = PS2NA.dropna()
PS2NA = PS2NA[(PS2NA['Ventas_norteamerica'] != 0)]
PS2NA['Ventas_norteamerica'] = PS2NA['Ventas_norteamerica'] * 1000000
PS2NA

,Plataforma,Ventas_norteamerica,Calificacion_critica
5,PS2,2660000.0,97.0
9,PS2,6990000.0,97.0
18,PS2,2080000.0,96.0
21,PS2,2450000.0,96.0
22,PS2,9430000.0,95.0
...,...,...,...
3403,PS2,170000.0,35.0
3404,PS2,110000.0,35.0
3407,PS2,70000.0,35.0
3427,PS2,40000.0,27.0


In [56]:
PS2EU = PS2EU.dropna()
PS2EU = PS2EU[(PS2EU['Ventas_europeas'] != 0)]
PS2EU['Ventas_europeas'] = PS2EU['Ventas_europeas'] * 1000000
PS2EU

,Plataforma,Ventas_europeas,Calificacion_critica
5,PS2,1290000.0,97.0
9,PS2,4510000.0,97.0
18,PS2,830000.0,96.0
21,PS2,2010000.0,96.0
22,PS2,400000.0,95.0
...,...,...,...
3403,PS2,130000.0,35.0
3404,PS2,90000.0,35.0
3407,PS2,60000.0,35.0
3427,PS2,30000.0,27.0


In [57]:
PS2JP = PS2JP.dropna()
PS2JP = PS2JP[(PS2JP['Ventas_japonesas'] != 0)]
PS2JP['Ventas_japonesas'] = PS2JP['Ventas_japonesas'] * 1000000
PS2JP

,Plataforma,Ventas_japonesas,Calificacion_critica
5,PS2,10000.0,97.0
9,PS2,300000.0,97.0
18,PS2,460000.0,96.0
21,PS2,870000.0,96.0
22,PS2,410000.0,95.0
...,...,...,...
3342,PS2,80000.0,50.0
3343,PS2,10000.0,50.0
3350,PS2,250000.0,49.0
3354,PS2,210000.0,48.0


In [58]:
PS4NA = PS4NA.dropna()
PS4NA = PS4NA[(PS4NA['Ventas_norteamerica'] != 0)]
PS4NA['Ventas_norteamerica'] = PS4NA['Ventas_norteamerica'] * 1000000
PS4NA

,Plataforma,Ventas_norteamerica,Calificacion_critica
7,PS4,3800000.0,97.0
70,PS4,1080000.0,93.0
75,PS4,1300000.0,93.0
90,PS4,1000000.0,92.0
100,PS4,960000.0,92.0
...,...,...,...
3301,PS4,130000.0,57.0
3311,PS4,20000.0,57.0
3318,PS4,70000.0,56.0
3324,PS4,40000.0,55.0


In [59]:
PS4EU = PS4EU.dropna()
PS4EU = PS4EU[(PS4EU['Ventas_europeas'] != 0)]
PS4EU['Ventas_europeas'] = PS4EU['Ventas_europeas'] * 1000000
PS4EU

,Plataforma,Ventas_europeas,Calificacion_critica
7,PS4,5810000.0,97.0
70,PS4,1350000.0,93.0
75,PS4,2070000.0,93.0
90,PS4,760000.0,92.0
100,PS4,2000000.0,92.0
...,...,...,...
3318,PS4,80000.0,56.0
3324,PS4,20000.0,55.0
3372,PS4,50000.0,42.0
3417,PS4,80000.0,32.0


In [60]:
PS4JP = PS4JP.dropna()
PS4JP = PS4JP[(PS4JP['Ventas_japonesas'] != 0)]
PS4JP['Ventas_japonesas'] = PS4JP['Ventas_japonesas'] * 1000000
PS4JP

,Plataforma,Ventas_japonesas,Calificacion_critica
7,PS4,360000.0,97.0
62,PS4,340000.0,93.0
70,PS4,480000.0,93.0
75,PS4,180000.0,93.0
90,PS4,260000.0,92.0
...,...,...,...
3122,PS4,40000.0,59.0
3175,PS4,20000.0,59.0
3235,PS4,30000.0,58.0
3251,PS4,20000.0,58.0


In [61]:
GCNA = GCNA.dropna()
GCNA = GCNA[(GCNA['Ventas_norteamerica'] != 0)]
GCNA['Ventas_norteamerica'] = GCNA['Ventas_norteamerica'] * 1000000
GCNA

,Plataforma,Ventas_norteamerica,Calificacion_critica
8,GC,1960000.0,97.0
14,GC,980000.0,96.0
17,GC,1150000.0,96.0
19,GC,2600000.0,96.0
53,GC,400000.0,94.0
...,...,...,...
3398,GC,90000.0,36.0
3401,GC,30000.0,35.0
3423,GC,60000.0,29.0
3426,GC,10000.0,27.0


In [63]:
GCEU = GCEU.dropna()
GCEU = GCEU[(GCEU['Ventas_europeas'] != 0)]
GCEU['Ventas_europeas'] = GCEU['Ventas_europeas'] * 1000000
GCEU

,Plataforma,Ventas_europeas,Calificacion_critica
8,GC,670000.0,97.0
14,GC,420000.0,96.0
17,GC,360000.0,96.0
19,GC,990000.0,96.0
53,GC,100000.0,94.0
...,...,...,...
3392,GC,10000.0,36.0
3396,GC,10000.0,36.0
3398,GC,20000.0,36.0
3401,GC,10000.0,35.0


In [64]:
GCJP = GCJP.dropna()
GCJP = GCJP[(GCJP['Ventas_japonesas'] != 0)]
GCJP['Ventas_japonesas'] = GCJP['Ventas_japonesas'] * 1000000
GCJP

,Plataforma,Ventas_japonesas,Calificacion_critica
8,GC,100000.0,97.0
14,GC,220000.0,96.0
17,GC,40000.0,96.0
19,GC,890000.0,96.0
67,GC,110000.0,93.0
72,GC,120000.0,93.0
102,GC,1390000.0,92.0
104,GC,870000.0,92.0
118,GC,70000.0,92.0
182,GC,30000.0,90.0


In [65]:
XBNA = XBNA.dropna()
XBNA = XBNA[(XBNA['Ventas_norteamerica'] != 0)]
XBNA['Ventas_norteamerica'] = XBNA['Ventas_norteamerica'] * 1000000
XBNA

,Plataforma,Ventas_norteamerica,Calificacion_critica
10,XB,4980000.0,97.0
27,XB,6820000.0,95.0
44,XB,70000.0,94.0
47,XB,840000.0,94.0
52,XB,1680000.0,94.0
...,...,...,...
3420,XB,40000.0,30.0
3424,XB,70000.0,28.0
3430,XB,20000.0,26.0
3433,XB,80000.0,25.0


In [66]:
XBEU = XBEU.dropna()
XBEU = XBEU[(XBEU['Ventas_europeas'] != 0)]
XBEU['Ventas_europeas'] = XBEU['Ventas_europeas'] * 1000000
XBEU

,Plataforma,Ventas_europeas,Calificacion_critica
10,XB,1300000.0,97.0
27,XB,1530000.0,95.0
44,XB,20000.0,94.0
47,XB,430000.0,94.0
52,XB,440000.0,94.0
...,...,...,...
3416,XB,10000.0,32.0
3420,XB,10000.0,30.0
3424,XB,20000.0,28.0
3430,XB,10000.0,26.0


In [67]:
XBJP = XBJP.dropna()
XBJP = XBJP[(XBJP['Ventas_japonesas'] != 0)]
XBJP['Ventas_japonesas'] = XBJP['Ventas_japonesas'] * 1000000
XBJP

,Plataforma,Ventas_japonesas,Calificacion_critica
10,XB,80000.0,97.0
27,XB,50000.0,95.0
106,XB,30000.0,92.0
142,XB,70000.0,91.0
187,XB,40000.0,90.0
239,XB,20000.0,89.0
307,XB,30000.0,88.0
352,XB,30000.0,87.0
355,XB,240000.0,87.0
494,XB,50000.0,85.0


In [68]:
GBANA = GBANA.dropna()
GBANA = GBANA[(GBANA['Ventas_norteamerica'] != 0)]
GBANA['Ventas_norteamerica'] = GBANA['Ventas_norteamerica'] * 1000000
GBANA

,Plataforma,Ventas_norteamerica,Calificacion_critica
28,GBA,1750000.0,95.0
32,GBA,850000.0,95.0
96,GBA,500000.0,92.0
119,GBA,1180000.0,92.0
153,GBA,200000.0,91.0
...,...,...,...
3405,GBA,20000.0,35.0
3411,GBA,70000.0,33.0
3422,GBA,100000.0,30.0
3429,GBA,50000.0,26.0


In [69]:
GBAEU = GBAEU.dropna()
GBAEU = GBAEU[(GBAEU['Ventas_europeas'] != 0)]
GBAEU['Ventas_europeas'] = GBAEU['Ventas_europeas'] * 1000000
GBAEU

,Plataforma,Ventas_europeas,Calificacion_critica
28,GBA,520000.0,95.0
32,GBA,310000.0,95.0
96,GBA,190000.0,92.0
119,GBA,270000.0,92.0
153,GBA,70000.0,91.0
...,...,...,...
3405,GBA,10000.0,35.0
3411,GBA,30000.0,33.0
3422,GBA,40000.0,30.0
3429,GBA,20000.0,26.0


In [70]:
GBAJP = GBAJP.dropna()
GBAJP = GBAJP[(GBAJP['Ventas_japonesas'] != 0)]
GBAJP['Ventas_japonesas'] = GBAJP['Ventas_japonesas'] * 1000000
GBAJP

,Plataforma,Ventas_japonesas,Calificacion_critica
28,GBA,330000.0,95.0
119,GBA,170000.0,92.0
120,GBA,280000.0,92.0
154,GBA,400000.0,91.0
157,GBA,50000.0,91.0
168,GBA,470000.0,90.0
235,GBA,220000.0,89.0
294,GBA,310000.0,88.0
310,GBA,540000.0,88.0
322,GBA,290000.0,88.0


In [71]:
PSVNA = PSVNA.dropna()
PSVNA = PSVNA[(PSVNA['Ventas_norteamerica'] != 0)]
PSVNA['Ventas_norteamerica'] = PSVNA['Ventas_norteamerica'] * 1000000
PSVNA

,Plataforma,Ventas_norteamerica,Calificacion_critica
298,PSV,130000.0,88.0
326,PSV,400000.0,88.0
362,PSV,30000.0,87.0
367,PSV,170000.0,87.0
416,PSV,210000.0,86.0
...,...,...,...
3088,PSV,110000.0,60.0
3095,PSV,210000.0,60.0
3113,PSV,80000.0,59.0
3196,PSV,130000.0,58.0


In [72]:
PSVEU = PSVEU.dropna()
PSVEU = PSVEU[(PSVEU['Ventas_europeas'] != 0)]
PSVEU['Ventas_europeas'] = PSVEU['Ventas_europeas'] * 1000000
PSVEU

,Plataforma,Ventas_europeas,Calificacion_critica
298,PSV,350000.0,88.0
326,PSV,720000.0,88.0
362,PSV,120000.0,87.0
367,PSV,390000.0,87.0
416,PSV,310000.0,86.0
...,...,...,...
3088,PSV,30000.0,60.0
3095,PSV,160000.0,60.0
3196,PSV,60000.0,58.0
3304,PSV,300000.0,57.0


In [73]:
PSVJP = PSVJP.dropna()
PSVJP = PSVJP[(PSVJP['Ventas_japonesas'] != 0)]
PSVJP['Ventas_japonesas'] = PSVJP['Ventas_japonesas'] * 1000000
PSVJP

,Plataforma,Ventas_japonesas,Calificacion_critica
326,PSV,20000.0,88.0
416,PSV,280000.0,86.0
543,PSV,90000.0,85.0
572,PSV,50000.0,84.0
688,PSV,10000.0,83.0
...,...,...,...
3088,PSV,60000.0,60.0
3095,PSV,20000.0,60.0
3113,PSV,40000.0,59.0
3211,PSV,10000.0,58.0


In [86]:
N64NA.to_csv('n64na.csv', index=False)
#PSNA.to_csv('psreal.csv', index=False)
#PS3NA.to_csv('ps3real.csv', index=False)
#DCNA.to_csv('n64real.csv', index=False)
#PS2NA.to_csv('ps2real.csv', index=False)
#PS4NA.to_csv('ps4real.csv', index=False)
#GCNA.to_csv('gcreal.csv', index=False)
#XBNA.to_csv('xbreal.csv', index=False)
#GBANA.to_csv('gbareal.csv', index=False)
#PSVNA.to_csv('psvreal.csv', index=False)

In [87]:
N64EU.to_csv('n64eu.csv', index=False)
#PSEU.to_csv('psreal.csv', index=False)
#PS3EU.to_csv('ps3real.csv', index=False)
#DCEU.to_csv('n64real.csv', index=False)
#PS2EU.to_csv('ps2real.csv', index=False)
#PS4EU.to_csv('ps4real.csv', index=False)
#GCEU.to_csv('gcreal.csv', index=False)
#XBEU.to_csv('xbreal.csv', index=False)
#GBAEU.to_csv('gbareal.csv', index=False)
#PSVEU.to_csv('psvreal.csv', index=False)

In [88]:
N64JP.to_csv('n64jp.csv', index=False)
#PSJP.to_csv('psreal.csv', index=False)
#PS3JP.to_csv('ps3real.csv', index=False)
#DCJP.to_csv('n64real.csv', index=False)
#PS2JP.to_csv('ps2real.csv', index=False)
#PS4JP.to_csv('ps4real.csv', index=False)
#GCJP.to_csv('gcreal.csv', index=False)
#XBJP.to_csv('xbreal.csv', index=False)
#GBAJP.to_csv('gbareal.csv', index=False)
#PSVJP.to_csv('psvreal.csv', index=False)